# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
S. Kraus  ->  S. Kraus  |  ['S. Kraus']
J. Li  ->  J. Li  |  ['J. Li']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Walter  ->  F. Walter  |  ['F. Walter']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


J. Liu  ->  J. Liu  |  ['J. Liu']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
Arxiv has 81 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.00829


extracting tarball to tmp_2411.00829... done.
Retrieving document from  https://arxiv.org/e-print/2411.01062


extracting tarball to tmp_2411.01062... done.


S. Kraus  ->  S. Kraus  |  ['S. Kraus']
Retrieving document from  https://arxiv.org/e-print/2411.01701


...

extracting tarball to tmp_2411.01960...

 done.
Retrieving document from  https://arxiv.org/e-print/2411.02352


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2411.01960/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authorlist' from 'tmp_2411.01960/authorlist.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2411.02352...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.01927-b31b1b.svg)](https://arxiv.org/abs/2411.01927) | **Euclid: The $r_{\rm b}$-$M_\ast$ relation as a function of redshift. I. The $5 \times 10^9 M_\odot$ black hole in NGC 1272**  |
|| R. Saglia, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-11-05*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Core ellipticals, massive early-type galaxies have an almost constant inner surface brightness profile. The size of the core region correlates with the mass of the finally merged black hole. Here we report the first Euclid-based dynamical mass determination of a supermassive black hole. We study the centre of NGC 1272, the second most luminous elliptical galaxy in the Perseus cluster, combining the Euclid VIS photometry coming from the Early Release Observations of the Perseus cluster with VIRUS spectroscopic observations at the Hobby-Eberly Telescope. The core of NGC 1272 is detected on the Euclid VIS image. Its size is $1.29\pm 0.07''$ or 0.45 kpc, determined by fitting PSF-convolved core-Sérsic and Nuker-law functions. The two-dimensional stellar kinematics of the galaxy is measured from the VIRUS spectra by deriving optimally regularized non-parametric line-of-sight velocity distributions. Dynamical models of the galaxy are constructed using our axisymmetric and triaxial Schwarzschild codes. We measure a black hole mass of $(5\pm3) \times 10^9 M_\odot$, in line with the expectation from the $M_{\rm BH}$-$r_{\rm b}$ correlation, but eight times larger than predicted by the $M_{\rm BH}$-$\sigma$ correlation (at $1.8\sigma$ significance). The core size, rather than the velocity dispersion, allows one to select galaxies harboring the most massive black holes. The spatial resolution, wide area coverage, and depth of the \Euclid (Wide and Deep) surveys allow us to find cores of passive galaxies larger than 2 kpc up to redshift 1.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.00829-b31b1b.svg)](https://arxiv.org/abs/2411.00829) | **A new multiple-arc model of the resonant Kuiper belt objects -- Plutinos**  |
|| Y. Chen, <mark>J. Li</mark> |
|*Appeared on*| *2024-11-05*|
|*Comments*| *arXiv admin note: substantial text overlap with arXiv:2305.18157*|
|**Abstract**|            To incorporate the gravitational influence of Kuiper belt objects (KBOs) in planetary ephemerides, uniform-ring models are commonly employed. In this paper, for representing the KBO population residing in Neptune's 2:3 mean motion resonance (MMR), known as the Plutinos, we introduce a three-arc model by considering their resonant characteristics. Each `arc' refers to a segment of the uniform ring and comprises an appropriate number of point masses. Then the total perturbation of Plutinos is numerically measured by the change in the Sun-Neptune distance ($\Delta d_{SN}$). We conduct a comprehensive investigation to take into account various azimuthal and radial distributions associated with the resonant amplitudes ($A$) and eccentricities ($e$) of Plutinos, respectively. The results show that over a 100-year period: (1) at the smallest $e=0.05$, the Sun-Neptune distance change $\Delta d_{SN}$ caused by Plutinos decreases significantly as $A$ reduces. It can deviate from the value of $\Delta d_{SN}$ obtained in the ring model by approximately 100 km; (2) as $e$ increases in the medium range of 0.1-0.2, the difference in $\Delta d_{SN}$ between the arc and ring models becomes increasingly significant; (3) at the largest $e\gtrsim0.25$, $\Delta d_{SN}$ can approach zero regardless of $A$, and the arc and ring models exhibit a substantial difference in $\Delta d_{SN}$, reaching up to 170 km. Then the applicability of our three-arc model is further verified by comparing it to the perturbations induced by observed Plutinos on the positions of both Neptune and Saturn. Moreover, the concept of the multiple-arc model, designed for Plutinos, can be easily extended to other MMRs densely populated by small bodies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.01823-b31b1b.svg)](https://arxiv.org/abs/2411.01823) | **Very High-energy Gamma-Ray Episodic Activity of Radio Galaxy NGC 1275 in 2022-2023 Measured with MACE**  |
|| S. Godambe, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2024-11-05*|
|*Comments*| *7 Pages, 5 Figures, and 1 Table*|
|**Abstract**|            The radio galaxy NGC 1275, located at the central region of Perseus cluster, is a well-known very high-energy (VHE) gamma-ray emitter. The Major Atmospheric Cherenkov Experiment Telescope has detected two distinct episodes of VHE (E > 80 GeV) gamma-ray emission from NGC 1275 during 2022 December and 2023 January. The second outburst, observed on 2023 January 10, was the more intense of the two, with flux reaching 58$\%$ of the Crab Nebula flux above 80 GeV. The differential energy spectrum measured between 80 GeV and 1.5 TeV can be described by a power law with a spectral index of $\Gamma = - 2.90 \pm 0.16_{stat}$ for both flaring events. The broadband spectral energy distribution derived from these flares, along with quasisimultaneous low-energy counterparts, suggests that the observed gamma-ray emission can be explained using a homogeneous single-zone synchrotron self-Compton model. The physical parameters derived from this model for both flaring states are similar. The intermediate state observed between two flaring episodes is explained by a lower Doppler factor or magnetic field, which subsequently returned to its previous value during the high-activity state observed on 2023 January 10.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.01960-b31b1b.svg)](https://arxiv.org/abs/2411.01960) | **The JCMT BISTRO Survey: The Magnetic Fields of the IC 348 Star-forming Region**  |
|| Y. Choi, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-11-05*|
|*Comments*| *Accepted for publication in ApJ. 21 pages, 12 figures*|
|**Abstract**|            We present 850 $\mu$m polarization observations of the IC 348 star-forming region in the Perseus molecular cloud as part of the B-fields In STar-forming Region Observation (BISTRO) survey. We study the magnetic properties of two cores (HH 211 MMS and IC 348 MMS) and a filamentary structure of IC 348. We find that the overall field tends to be more perpendicular than parallel to the filamentary structure of the region. The polarization fraction decreases with intensity, and we estimate the trend by power-law and the mean of the Rice distribution fittings. The power indices for the cores are much smaller than 1, indicative of possible grain growth to micron size in the cores. We also measure the magnetic field strengths of the two cores and the filamentary area separately by applying the Davis-Chandrasekhar-Fermi method and its alternative version for compressed medium. The estimated mass-to-flux ratios are 0.45-2.20 and 0.63-2.76 for HH 211 MMS and IC 348 MMS, respectively, while the ratios for the filament is 0.33-1.50. This result may suggest that the transition from subcritical to supercritical conditions occurs at the core scale ($\sim$ 0.05 pc) in the region. In addition, we study the energy balance of the cores and find that the relative strength of turbulence to the magnetic field tends to be stronger for IC 348 MMS than HH 211 MMS. The result could potentially explain the different configurations inside the two cores: a single protostellar system in HH 211 MMS and multiple protostars in IC 348 MMS.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.02352-b31b1b.svg)](https://arxiv.org/abs/2411.02352) | **Virgo Filaments IV: Using WISE to Measure the Modification of Star-Forming Disks in the Extended Regions Around the Virgo Cluster**  |
|| K. Conger, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-11-05*|
|*Comments*| *21 pages, 11 figures. Resubmitted second time to ApJ with only minor revision requested*|
|**Abstract**|            Recent theoretical work and targeted observational studies suggest that filaments are sites of galaxy preprocessing. The aim of the WISESize project is to directly probe galaxies over the full range of environments to quantify and characterize extrinsic galaxy quenching in the local Universe. In this paper, we use GALFIT to measure the infrared 12$\mu$m ($R_{12}$) and 3.4$\mu$m ($R_{3.4}$) effective radii of 603 late-type galaxies in and surrounding the Virgo cluster. We find that Virgo cluster galaxies show smaller star-forming disks relative to their field counterparts at the $2.5\sigma$ level, while filament galaxies show smaller star-forming disks to almost $1.5\sigma$. Our data, therefore, show that cluster galaxies experience significant effects on their star-forming disks prior to their final quenching period. There is also tentative support for the hypothesis that galaxies are preprocessed in filamentary regions surrounding clusters. On the other hand, galaxies belonging to rich groups and poor groups do not differ significantly from those in the field. We additionally find hints of a positive correlation between stellar mass and size ratio for both rich group and filament galaxies, though the uncertainties on these data are consistent with no correlation. We compare our size measurements with the predictions from two variants of a state-of-the-art semi-analytic model (SAM), one which includes starvation and the other incorporating both starvation and ram-pressure stripping (RPS). Our data appear to disfavor the SAM, which includes RPS for the rich group, filament, and cluster samples, which contributes to improved constraints for general models of galaxy quenching.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.01062-b31b1b.svg)](https://arxiv.org/abs/2411.01062) | **Visual Orbits of Wolf-Rayet Stars II: The Orbit of the Nitrogen-Rich WR Binary WR 138 measured with the CHARA Array**  |
|| A. Holdsworth, et al. -- incl., <mark>S. Kraus</mark> |
|*Appeared on*| *2024-11-05*|
|*Comments*| *accepted to ApJ*|
|**Abstract**|            Classical Wolf-Rayet stars are descendants of massive OB-type stars that have lost their hydrogen-rich envelopes, and are in the final stages of stellar evolution, possibly exploding as type Ib/c supernovae. It is understood that the mechanisms driving this mass-loss are either strong stellar winds and or binary interactions, so intense studies of these binaries including their evolution can tell us about the importance of the two pathways in WR formation. WR 138 (HD 193077) has a period of just over 4 years and was previously reported to be resolved through interferometry. We report on new interferometric data combined with spectroscopic radial velocities in order to provide a three-dimensional orbit of the system. The precision on our parameters tend to be about an order of magnitude better than previous spectroscopic techniques. These measurements provide masses of the stars, namely $M_{\rm WR} = 13.93\pm1.49M_{\odot}$ and $M_{\rm O} = 26.28\pm1.71M_{\odot}$. The derived orbital parallax agrees with the parallax from \textit{Gaia}, namely with a distance of 2.13 kpc. We compare the system's orbit to models from BPASS, showing that the system likely may have been formed with little interaction but could have formed through some binary interactions either following or at the start of a red supergiant phase, but with the most likely scenario occurring as the red supergiant phase starts for a $\sim 40M_\odot$ star.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.01701-b31b1b.svg)](https://arxiv.org/abs/2411.01701) | **Constraining the excitation of molecular gas in Two Quasar-Starburst Systems at $z \sim 6$**  |
|| F. Xu, et al. -- incl., <mark>J. Li</mark>, <mark>E. Bañados</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2024-11-05*|
|*Comments*| *19 pages, 9 figures, accepted for publication in ApJ*|
|**Abstract**|            We present NOrthern Extended Millimeter Array observations of CO(8-7), (9-8), and (10-9) lines, as well as the underlying continuum for two far-infrared luminous quasars: SDSS J2054-0005 at $\rm z=6.0389$ and SDSS J0129-0035 at $\rm z=5.7788$. Both quasars were previously detected in CO (2-1) and (6-5) transitions, making them candidates for studying the CO Spectral Line Energy Distribution (SLED) of quasars at $z \sim 6$. Utilizing the radiative transfer code CLOUDY, we fit the CO SLED with two heating mechanisms, including the photo-dissociation region (PDR) and X-ray-dominated region (XDR) for both objects. The CO SLEDs can be fitted by either a dense PDR component with an extremely strong far-ultraviolet radiation field (gas density $ n_{\rm H} \sim 10^6 \, \rm cm^{-3}$ and field strength $G_0 \gtrsim 10^6$) or a two-component model including a PDR and an XDR. However, the line ratios, including \tir and previous \cii and \ci measurements, argue against a very high PDR radiation field strength. Thus, the results prefer a PDR+XDR origin for the CO SLED. The excitation of the high-J CO lines in both objects is likely dominated by the central AGN. We then check the CO (9-8)-to-(6-5) line luminosity ratio $r_{96}$ for all $z \sim 6$ quasars with available CO SLEDs (seven in total) and find that there are no clear correlations between $r_{96}$ and both \fir and the AGN UV luminosities. This further demonstrates the complexity of the CO excitation powered by both the AGN and nuclear star formation in these young quasar host galaxies.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.01927/./newphot1272trim.png', 'tmp_2411.01927/./pq_itvls_all_slz_xaxis_axi_trim.png', 'tmp_2411.01927/./NGC1272_kinematic_map_trimmed.png']
copying  tmp_2411.01927/./newphot1272trim.png to _build/html/
copying  tmp_2411.01927/./pq_itvls_all_slz_xaxis_axi_trim.png to _build/html/
copying  tmp_2411.01927/./NGC1272_kinematic_map_trimmed.png to _build/html/
exported in  _build/html/2411.01927.md
    + _build/html/tmp_2411.01927/./newphot1272trim.png
    + _build/html/tmp_2411.01927/./pq_itvls_all_slz_xaxis_axi_trim.png
    + _build/html/tmp_2411.01927/./NGC1272_kinematic_map_trimmed.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# $\Euclid$: The $r_{\rm b}$-$M_\ast$ relation as a function of redshift. I. The $5\expo{9}\solarmass$ black hole in NGC 1272$\thanks{This paper is    published on       behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.01927-b31b1b.svg)](https://arxiv.org/abs/2411.01927)<mark>Appeared on: 2024-11-05</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

R. Saglia, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Core ellipticals, massive    early-type galaxies with almost constant inner surface brightness    profiles, are the results of dry mergers. During these events a    binary black hole is formed, destroying the original cuspy central    regions of the merging objects and scattering stars that are not on    tangential orbits. The size of the emerging core correlates with    the mass of the finally merged black hole. Therefore, the    determination of the size of the core of massive early type    galaxies provides key insights not only on the mass of the black    hole, but also on the origin and evolution of these objects.    In this work we report the first $\Euclid$ -based dynamical mass determination of a supermassive black    hole.  We perform it by studying the centre of NGC 1272, the    second most luminous elliptical galaxy in the Perseus cluster,    combining the $\Euclid$ VIS photometry coming from the Early Release    Observations of the Perseus cluster with VIRUS spectroscopic    observations at the Hobby-Eberly Telescope.    The core of NGC 1272 is detected    on the $\Euclid$ VIS image. Its size is $1\arcsecf29\pm 0\arcsecf07$ or 0.45 $\kiloparsec$ , determined by    fitting PSF-convolved core-Sérsic and Nuker-law functions. We    deproject the surface brightness profile of the galaxy, finding    that the galaxy is axisymmetric and nearly spherical. The    two-dimensional stellar kinematics of the galaxy is measured from    the VIRUS spectra by deriving optimally regularized non-parametric    line-of-sight velocity distributions. Dynamical models of the    galaxy are constructed using our axisymmetric and triaxial    Schwarzschild codes.    We measure a black hole mass of $(5\pm3)\expo{9}\solarmass$ ,    in line with the expectation from the $M_{\rm BH}$ - $r_{\rm b}$ correlation, but eight times larger than    predicted by the $M_{\rm BH}$ - $\sigma$ correlation (at $1.8\sigma$ significance).    The core size, rather than the velocity dispersion, allows one to    select galaxies harboring the most massive black holes. The    spatial resolution, wide area coverage, and depth of the $\Euclid$ (Wide and Deep) surveys allow us to find cores of passive galaxies    larger than 2 $\kiloparsec$ up to redshift 1.

</div>

<div id="div_fig1">

<img src="tmp_2411.01927/./newphot1272trim.png" alt="Fig3" width="100%"/>

**Figure 3. -** *Top Left:* surface brightness profile of NGC 1272,
  measured from the \Euclid VIS image, calibrated to the $V-$band
  and corrected for Galactic absorption and cosmological dimming,
  as a function of the 1/4 power of the semi-major distance
  $A$ on the sky in arcsec. *Bottom Left:* the difference
  $\Delta$_ SB_ between the surface brightness profile of NGC 1272 and the
  surface brightness of the core-Sérsic+Sérsic model.
  *Right:* ellipticity $1-B/A$, where $B$ is the semi-minor
  axis length on the sky (top) and PA (bottom) as a function of the 1/4
  power of $A$.  The solid red lines show the core-Sérsic+Sérsic
  model. The dashed lines show its core radius. The blue line shows
  the ellipticity profile of the axisymmetric deprojection. The green
  line shows the PA profile of the triaxial deprojection. (*Fig_photoN1272*)

</div>
<div id="div_fig2">

<img src="tmp_2411.01927/./pq_itvls_all_slz_xaxis_axi_trim.png" alt="Fig5" width="100%"/>

**Figure 5. -** Profiles of $p(r)$ and $q(r)$(where $p=b/a$, $q=c/a$ and $a$, $b$ and $c$ are the major,
intermediate and minor semi-axis of the galaxy) derived from the axisymmetric (dotted), triaxial (full line), the spherical (dashed line), and prolate (dashed-dotted line) deprojections
of the galaxy, as a function of the distance $r$ from the center.
The red and blue shaded areas show the whole range of
      allowed deprojections with ${\rm RMS}\le 1.2\times{\rm RMS_{min}}$ ([Saglia, et. al 2020](), [Saglia and Thomas 2022](), [Neureiter, et. al 2022]()) . (*Fig_pqprofiles*)

</div>
<div id="div_fig3">

<img src="tmp_2411.01927/./NGC1272_kinematic_map_trimmed.png" alt="Fig11" width="100%"/>

**Figure 11. -** Two-dimensional stellar kinematics of NGC 1272.
      The horizontal and vertical dashed lines show the major and minor
      axes of the galaxy, respectively. North is up and east is to the left. (*Fig_2dimKinN1272*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.01927"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

239  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
